In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [3]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install pyspark
!pip install pyspark

# Set up Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkSQL") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import current_date, expr



# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Path to the CSV files in Google Drive
show_data_path = '/content/drive/My Drive/showData.csv'
movie_data_path = '/content/drive/My Drive/movieData.csv'

In [7]:
# Load the CSV files into DataFrames
show_df = pd.read_csv(show_data_path)
movie_df = pd.read_csv(movie_data_path)


In [8]:
# Display the first few rows of showData.csv
show_df.head()

,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score
0,0,ts300399,Five Came Back: The Reference Films,1945,TV-MA,51,['documentation'],['US'],1.0,NaN
1,1,ts22164,Monty Python's Flying Circus,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8
2,2,ts45948,Monty Python's Fliegender Zirkus,1972,TV-MA,43,['comedy'],[],1.0,8.1
3,3,ts20681,Seinfeld,1989,TV-PG,24,['comedy'],['US'],9.0,8.9
4,4,ts22082,Knight Rider,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9


In [9]:
# Display the first few rows of movieData.csv
movie_df.head()

,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,imdb_score
0,0,tm84618,Taxi Driver,1976,R,114,"['drama', 'crime']",['US'],8.2
1,1,tm154986,Deliverance,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],7.7
2,2,tm127384,Monty Python and the Holy Grail,1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],8.2
3,3,tm120801,The Dirty Dozen,1967,NaN,150,"['war', 'action']","['GB', 'US']",7.7
4,4,tm70993,Life of Brian,1979,R,94,['comedy'],['GB'],8.0


In [10]:
# Replace NaN IMDb scores with None in show_df
show_df['imdb_score'] = show_df['imdb_score'].apply(lambda x: None if pd.isnull(x) else x)

# Replace NaN IMDb scores with None in movie_df
movie_df['imdb_score'] = movie_df['imdb_score'].apply(lambda x: None if pd.isnull(x) else x)

In [11]:
# Convert cleaned Pandas DataFrames to Spark DataFrames
spark_show_df_cleaned = spark.createDataFrame(show_df)
spark_movie_df_cleaned = spark.createDataFrame(movie_df)

# Create temporary views for the cleaned Spark DataFrames
spark_show_df_cleaned.createOrReplaceTempView("showData_cleaned")
spark_movie_df_cleaned.createOrReplaceTempView("movieData_cleaned")

In [12]:
# Display the schema of the Spark DataFrame
spark_show_df_cleaned.printSchema()



root
 |-- Unnamed: 0: long (nullable = true)
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: long (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- seasons: double (nullable = true)
 |-- imdb_score: double (nullable = true)



In [13]:
# Display the schema of the Spark DataFrame
spark_movie_df_cleaned.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: long (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- imdb_score: double (nullable = true)



# What are the top-rated TV shows and movies based on IMDb scores?

In [14]:
# Finding top rated tv shows and movies, excluding rows with NaN IMDb scores
top_rated = spark.sql("""
    SELECT 'TV Show' AS type, title, imdb_score
    FROM showData_cleaned
    WHERE imdb_score IS NOT NULL AND NOT isnan(imdb_score)
    UNION ALL
    SELECT 'Movie' AS type, title, imdb_score
    FROM movieData_cleaned
    WHERE imdb_score IS NOT NULL AND NOT isnan(imdb_score)
    ORDER BY imdb_score DESC
    LIMIT 10
""")
top_rated.show()


+-------+--------------------+----------+
|   type|               title|imdb_score|
+-------+--------------------+----------+
|TV Show|            #ABtalks|       9.6|
|TV Show|        Breaking Bad|       9.5|
|TV Show|            Khawatir|       9.5|
|TV Show|          Our Planet|       9.3|
|TV Show|Avatar: The Last ...|       9.3|
|TV Show|          Reply 1988|       9.2|
|  Movie|               Major|       9.1|
|  Movie|Chhota Bheem & Kr...|       9.1|
|TV Show|        Kota Factory|       9.1|
|TV Show|           My Mister|       9.1|
+-------+--------------------+----------+



# Which genres are most common among TV shows and movies?

In [15]:
result = spark.sql("""
    SELECT type, trim('\"' FROM genre) AS genre, COUNT(*) AS count
    FROM
      (SELECT 'TV Show' AS type, explode(split(trim('[]' FROM genres), ', ')) AS genre FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(trim('[]' FROM genres), ', ')) AS genre FROM movieData_cleaned)
    WHERE genre != ""
    GROUP BY type, genre
    ORDER BY count DESC
""")
# Show the result
result.show()

+-------+---------------+-----+
|   type|          genre|count|
+-------+---------------+-----+
|  Movie|        'drama'| 1876|
|  Movie|       'comedy'| 1571|
|TV Show|        'drama'| 1092|
|  Movie|     'thriller'|  825|
|TV Show|       'comedy'|  754|
|  Movie|       'action'|  718|
|  Movie|      'romance'|  698|
|  Movie|'documentation'|  611|
|  Movie|        'crime'|  545|
|TV Show|       'action'|  439|
|TV Show|    'animation'|  423|
|TV Show|     'thriller'|  403|
|TV Show|        'crime'|  391|
|TV Show|        'scifi'|  385|
|  Movie|       'family'|  351|
|  Movie|     'european'|  344|
|TV Show|'documentation'|  341|
|TV Show|       'family'|  331|
|TV Show|      'fantasy'|  315|
|  Movie|      'fantasy'|  315|
+-------+---------------+-----+
only showing top 20 rows



# How does the distribution of IMDb scores differ between TV shows and movies?

In [16]:
from pyspark.sql.functions import col

# Calculate the average IMDb score for TV shows, excluding NaN values
tv_show_avg = spark.sql("""
    SELECT AVG(imdb_score) AS avg_imdb_score
    FROM showData_cleaned
    WHERE imdb_score IS NOT NULL AND NOT isnan(imdb_score)
""")
tv_show_avg_score = tv_show_avg.collect()[0]["avg_imdb_score"]

# Calculate the average IMDb score for movies, excluding NaN values
movie_avg = spark.sql("""
    SELECT AVG(imdb_score) AS avg_imdb_score
    FROM movieData_cleaned
    WHERE imdb_score IS NOT NULL AND NOT isnan(imdb_score)
""")
movie_avg_score = movie_avg.collect()[0]["avg_imdb_score"]

# Display the results
print("Average IMDb score for TV shows:", tv_show_avg_score)
print("Average IMDb score for movies:", movie_avg_score)


Average IMDb score for TV shows: 6.977926766374413
Average IMDb score for movies: 6.246748323126284


In [17]:
# Calculate the average IMDb score for TV shows and movies, excluding NaN values
result = spark.sql("""
    SELECT 'All' AS type,
           FORMAT_NUMBER(AVG(imdb_score), 2) AS avg_imdb_score
    FROM
    (
        SELECT imdb_score FROM showData_cleaned WHERE NOT isnan(imdb_score)
        UNION ALL
        SELECT imdb_score FROM movieData_cleaned WHERE NOT isnan(imdb_score)
    )
""")
result.show()


+----+--------------+
|type|avg_imdb_score|
+----+--------------+
| All|          6.51|
+----+--------------+



# What is the average runtime of TV shows and movies?

In [18]:
result = spark.sql("""
    SELECT 'TV Show' AS type, AVG(runtime) AS avg_runtime
    FROM showData_cleaned
    WHERE runtime IS NOT NULL
    UNION ALL
    SELECT 'Movie' AS type, AVG(runtime) AS avg_runtime
    FROM movieData_cleaned
    WHERE runtime IS NOT NULL
    GROUP BY type
""")
result.show()


+-------+------------------+
|   type|       avg_runtime|
+-------+------------------+
|TV Show|38.978157644824314|
|  Movie| 98.21367521367522|
+-------+------------------+



# How has the annual release pattern of TV shows and movies evolved over the span of 10 years, from 2013 to 2022?

In [19]:
# Query to count the number of TV shows and movies released each year
result = spark.sql("""
    SELECT type, release_year, COUNT(*) AS count
    FROM
      (SELECT 'TV Show' AS type, release_year FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, release_year FROM movieData_cleaned)
    WHERE release_year IS NOT NULL
    GROUP BY type, release_year
    ORDER BY release_year DESC, type DESC
    LIMIT 20
""")

# Show the result
result.show()


+-------+------------+-----+
|   type|release_year|count|
+-------+------------+-----+
|TV Show|        2022|  171|
|  Movie|        2022|  200|
|TV Show|        2021|  314|
|  Movie|        2021|  473|
|TV Show|        2020|  314|
|  Movie|        2020|  500|
|TV Show|        2019|  311|
|  Movie|        2019|  525|
|TV Show|        2018|  300|
|  Movie|        2018|  473|
|TV Show|        2017|  179|
|  Movie|        2017|  384|
|TV Show|        2016|  134|
|  Movie|        2016|  228|
|TV Show|        2015|   89|
|  Movie|        2015|  134|
|TV Show|        2014|   44|
|  Movie|        2014|  109|
|TV Show|        2013|   35|
|  Movie|        2013|  100|
+-------+------------+-----+



# Which countries produce the most TV shows and movies?


In [20]:
# Query to count the number of TV shows and movies produced by each country
result = spark.sql("""
    SELECT production_country,
           SUM(CASE WHEN type = 'TV Show' THEN 1 ELSE 0 END) AS tv_count,
           SUM(CASE WHEN type = 'Movie' THEN 1 ELSE 0 END) AS movie_count,
           COUNT(*) AS total_count
    FROM
      (SELECT 'TV Show' AS type, explode(split(production_countries, ', ')) AS production_country FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(production_countries, ', ')) AS production_country FROM movieData_cleaned)
    WHERE production_country != "" AND production_country NOT LIKE "%[%" AND production_country NOT LIKE "%]%"
    GROUP BY production_country
    ORDER BY total_count DESC
""")

# Show the result
result.show()


+------------------+--------+-----------+-----------+
|production_country|tv_count|movie_count|total_count|
+------------------+--------+-----------+-----------+
|              'US'|       3|         36|         39|
|              'GB'|       0|         34|         34|
|              'FR'|       2|         29|         31|
|              'DE'|       0|         20|         20|
|              'CA'|       0|         11|         11|
|              'DK'|       0|         11|         11|
|              'NL'|       0|          8|          8|
|              'CH'|       0|          7|          7|
|              'CN'|       0|          6|          6|
|              'ES'|       0|          5|          5|
|              'BE'|       0|          5|          5|
|              'SE'|       0|          5|          5|
|              'MX'|       0|          5|          5|
|              'JP'|       1|          3|          4|
|              'AU'|       1|          3|          4|
|              'LB'|       0

# Which age certification (e.g., TV-MA, PG-13) is most common among TV shows and movies?

In [21]:
result = spark.sql("""
    SELECT age_certification, COUNT(*) AS count
    FROM
      (SELECT age_certification FROM showData_cleaned WHERE age_certification IS NOT NULL AND age_certification != '' AND age_certification != 'NaN'
       UNION ALL
       SELECT age_certification FROM movieData_cleaned WHERE age_certification IS NOT NULL AND age_certification != '' AND age_certification != 'NaN')
    GROUP BY age_certification
    ORDER BY count DESC
""")
result.show()


+-----------------+-----+
|age_certification|count|
+-----------------+-----+
|            TV-MA|  883|
|                R|  556|
|            TV-14|  474|
|            PG-13|  451|
|               PG|  233|
|            TV-PG|  188|
|                G|  124|
|            TV-Y7|  120|
|             TV-Y|  107|
|             TV-G|   79|
|            NC-17|   16|
+-----------------+-----+



# What is the average number of seasons for TV shows in each age certification category?

In [22]:
result = spark.sql("""
    SELECT age_certification, AVG(seasons) AS avg_seasons
    FROM showData_cleaned
    WHERE age_certification IS NOT NULL AND age_certification != 'NaN'
    GROUP BY age_certification
    ORDER BY avg_seasons DESC
""")

result.show()


+-----------------+------------------+
|age_certification|       avg_seasons|
+-----------------+------------------+
|             TV-G|3.6202531645569622|
|            TV-Y7|3.3916666666666666|
|             TV-Y|3.0373831775700935|
|            TV-PG| 2.702127659574468|
|            TV-14| 2.413502109704641|
|            TV-MA|1.7066817667044167|
+-----------------+------------------+



# How does the average IMDb score vary by production country for TV shows and movies?

In [23]:
result = spark.sql("""
    SELECT type, production_country, AVG(imdb_score) AS avg_imdb_score
    FROM
      (SELECT 'TV Show' AS type, explode(split(production_countries, ', ')) AS production_country, imdb_score FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(production_countries, ', ')) AS production_country, imdb_score FROM movieData_cleaned)
    WHERE production_country != "" AND production_country NOT LIKE "%[%" AND production_country NOT LIKE "%]%" AND imdb_score IS NOT NULL AND NOT isnan(imdb_score)
    GROUP BY type, production_country
    ORDER BY avg_imdb_score DESC
""")

result.show()


+-------+------------------+------------------+
|   type|production_country|    avg_imdb_score|
+-------+------------------+------------------+
|  Movie|              'EG'|               8.1|
|  Movie|              'BS'|               8.0|
|  Movie|              'MW'|               7.6|
|  Movie|              'NO'|              7.35|
|  Movie|              'LB'|               7.3|
|  Movie|              'BR'|               7.2|
|  Movie|              'AE'|               7.1|
|  Movie|              'UY'|               7.1|
|  Movie|              'ES'|7.0200000000000005|
|  Movie|              'GH'|               7.0|
|  Movie|              'ZA'| 6.949999999999999|
|  Movie|              'CH'| 6.914285714285714|
|  Movie|              'MX'|              6.88|
|TV Show|              'FR'|              6.85|
|  Movie|              'IS'| 6.833333333333333|
|  Movie|              'SE'|              6.82|
|  Movie|              'IR'|               6.8|
|  Movie|              'JO'|            